## Set Up

In [ ]:
# !pip install --upgrade typing_extensions
# !pip install langchain langchain-community langchain-openai langchain-core chromadb openai sentence-transformers langchain-huggingface
# !pip install python-dotenv

In [1]:
import shutil

zip_path = "/Users/muhammetsantas/Downloads/EDUCATION/POWERCODERS/PERSONAL_PROJECTS/chatbot/project-25-2-scrum-team-data/vector_pipeline/chroma_db.zip"
extract_path = "/Users/muhammetsantas/Downloads/EDUCATION/POWERCODERS/PERSONAL_PROJECTS/chatbot/chroma_db"

shutil.unpack_archive(zip_path, extract_path)

## Import Libraries

In [2]:
import os
import zipfile
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip freeze | grep langchain

langchain==1.1.0
langchain-classic==1.0.0
langchain-community==0.4.1
langchain-core==1.1.0
langchain-huggingface==1.1.0
langchain-openai==1.1.0
langchain-text-splitters==1.0.0


## Create Prompt

In [3]:
prompt_template = """You are a product knowledge assistant specialized in musical instruments and music-related equipment.
Your job is to answer user questions using ONLY the information contained in the retrieved context documents.

The dataset you are working with contains:
- product title, description, features, categories
- product metadata (store, color, rating, rating count, price, etc.)

### Your Rules:
1. Use ONLY the retrieved context to answer questions.  
2. Do NOT invent product details, specifications, or metadata that are not present in the context.  
3. If the context does not contain the required information, say:  
   "The provided product information does not include this detail."
4. If the question is about comparing products, create a clear comparison using only the available data.
5. Summaries must be concise and factual.
6. Preserve any numerical values exactly as they appear in the context.
7. If the user asks about availability or stock, respond with:  
   "This dataset does not include real-time availability information."
8. When the question is unclear, ask for clarification.
9. If metadata is available in the context (e.g., store, color, rating), include it in your answer.
10. NEVER output raw JSON or database structures—respond in clean natural language.

### Response Format:
- Always provide a short, direct answer first.
- If relevant, include a structured breakdown:
  - **Key Features**
  - **Specifications / Metadata**
  - **Summary**

### Context:
{context}

### User Question:
{question}

### Final Answer:
(Your answer here)
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

print("Prompt was created successfully.")

Prompt was created successfully.


## BGE EMbeddings and Vector DB Load

In [4]:
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

print(f"Embeddings model was loaded successfully: {model_name}")

Embeddings model was loaded successfully: BAAI/bge-base-en-v1.5


In [17]:
vdb_path = "/Users/muhammetsantas/Downloads/EDUCATION/POWERCODERS/PERSONAL_PROJECTS/chatbot/project-25-2-scrum-team-data/chroma_db"

# Load Chroma DB
vectorstore = Chroma(
    persist_directory=vdb_path,
    embedding_function=embeddings,
    collection_metadata={"hnsw:space": "cosine"}
)

print(f"Vector store was loaded successfully. Total document count: {vectorstore._collection.count()}")

Vector store was loaded successfully. Total document count: 6401


## LLM Model

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env", override=True)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if OPENROUTER_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENROUTER_API_KEY
    os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
    print("✅ API key was loaded successfully!")
    print(f"📍 .env file location: ../../.env")
else:
    print("❌ API key couldn't be found. Please check the .env file.")
    print(f"🔍 Searched location: ../../.env")

✅ API key was loaded successfully!
📍 .env file location: ../../.env


In [ ]:
OPENROUTER_API_KEY

In [ ]:
llm = ChatOpenAI(
    model="google/gemma-3-27b-it:free",
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.7,
    max_tokens=2000
)

print(f"LLM model was set successfully {llm.model_name}...")

LLM model was set successfully google/gemma-3-27b-it:free...


## Create RAG Chain

In [21]:
def format_docs(docs):
    formatted = []
    for i, doc in enumerate(docs):
        formatted.append(
            f"### Document {i+1}\n"
            f"Content:\n{doc.page_content}\n\n"
            f"Metadata:\n{doc.metadata}\n"
        )
    return "\n\n".join(formatted)


retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | PROMPT
    | llm
    | StrOutputParser()
)

print("RAG chain was created successfully!")

RAG chain was created successfully!


## Test

In [22]:
def ask_question(question):
    response = qa_chain.invoke(question)
    print(f"Query: {question}")
    print(f"\nResponse: {response}")
    print("\n" + "="*70 + "\n")
    return response

# Sample questions to test the setup
questions = [
    "What is the price of the most rated guitar?",
    "Which guitar has the best reviews?",
    "What kind of guitars are available for beginners?"
]

#for q in questions:
#    ask_question(q)

In [23]:
query = "Show me the top 5 highest rated electric guitars for beginners under $500. Use the metadata if available."

response = qa_chain.invoke(query)

print("Question:", query)
print("\nAnswer:", response)

Question: Show me the top 5 highest rated electric guitars for beginners under $500. Use the metadata if available.

Answer: 
Here are the top 5 highest-rated electric guitars for beginners under $500, based on the provided information:

1.  **LyxPro Left Hand 39 Inch Electric Guitar and Starter Kit** (Rating: 4.4, Rating Count: 9358, Price: $149.99, Color: Natural, Store: LyxPro)
2.  **Stedman Pro Beginner Series 39-Inch Electric Guitar** (Rating: 4.3, Rating Count: 861, Price: $124.95, Color: Black, Store: Stedman Pro)
3.  **Smartxchoices 39’’ Electric Guitar** (Rating: 4.4, Rating Count: 629, Price: $96.99, Color: Blue, Store: Smartxchoices)
4.  **Davison Guitars 39" Electric Guitar** (Rating: 4.3, Rating Count: 2097, Price: $119.99, Color: Black, Store: Davison Guitars)
5.  **COLUBER CABLE Electric Guitar Beginner Kit** (Rating: 4.4, Rating Count: 1119, Price: $149.99, Color: Red, Store: COLUBER CABLE)

**Key Features (across products):**

*   Full-size guitars (39 inches is common

In [12]:
import pandas as pd

raw_df = pd.read_pickle("../data/processed_product.pkl")
raw_df

,rating,rating_count,price,store,product_id,color,combined_text
0,4.8,100615,6.99,Ernie Ball,B0BSGM6CQ9,Mondo (10.5-52),Product Title: Ernie Ball Mondo Slinky Nickelw...
1,4.4,79425,29.99,BONAOK,B09W4F2X6S,Black Gold,Product Title: BONAOK Wireless Bluetooth Karao...
2,4.0,68708,14.95,BOYA,B076B8G5D8,Black,Product Title: Boya BYM1 by Shotgun Video Micr...
3,4.7,60133,10.99,D'Addario,B0BTC9YJ2W,Bronze,Product Title: D'Addario Guitar Strings - Phos...
4,4.8,55452,17.75,Amazon Basics,B018FCZKR2,Not Specified,Product Title: Amazon Basics Adjustable Guitar...
...,...,...,...,...,...,...,...
3729,4.4,548,31.99,Shamanicshop,B017NGQWA0,Brown,Product Title: Jew's Harp by P.Potkin in woode...
3730,4.3,548,13.32,Musiclily,B00WHCNE0M,Gold,Product Title: Musiclily 42mm Guitar Metal Loc...
3731,4.3,548,18.99,SumDirect,B00M35GOP0,White,Product Title: SumDirect 9 LED Music Stand Lig...
3732,4.6,547,189.99,BOSS,B086QNCLQ8,Silver,Product Title: Boss SYB-5 Bass Synthesizer Ped...


In [30]:
docs = retriever.invoke(query)
for i, d in enumerate(docs):
    print(f"---- DOCUMENT {i} ----")
    print(d.page_content)
    print("METADATA:", d.metadata, "\n")


---- DOCUMENT 0 ----
Description: Smartxchoices 39’’ Electric Guitar is a brand new functional beginner guitar. High quality and beautiful surface make it an ideal choice for you. It is lightweight and easy to carry with the included guitar bag. Made of all wood construction with steel strings, delivers a smooth tone and creates bright sound. All accessories you need to start playing the guitar right away are included. This blue guitar is perfect for guitar beginners and would make a decent gift. Kids, teens, and adults can learn and creat wonderful music with this full size guitar kit !: Features:: ♬ 39" Full Size Blue Electric Guitar(Right-handed),ideal for beginners,aspiring musicians and expert guitar heroes.  ♬ Ultra-Thin slick design for easy control during playing, either sitting or standing up. ♬ Fantastic paint and smooth finish, great decoration in your room. ♬ The natural, bright sound complements the warm sounds of the solid wood. ♬ Comes with a 10W amp and carry bag for ea

In [40]:
print(raw_df.combined_text[3213])

Product Title: Smartxchoices 39" Electric Guitar Beginner Kit Full Size Blue Guitar with 10W Amp, Case and Accessories Pack for Starter
Categories: Musical Instruments, Guitars, Electric Guitars, Beginner Kits
Features: 【All-wood Construction】 Solid hardwood body with steel strings, the natural, bright sound complements the warm sounds of the solid wood. High quality rosewood fretboard and premium maple neck, durable and attractive. 【User-friendly Design】 Ultra-Thin slick design for easy control during playing, either sitting or standing up. The amp has portable handle, easy to carry and move. Comes with a nylon carrying case, perfect for traveling musicians and easy storage. 【Nice Look】Classic Design with stylish finish, looks so cool. Ideal for rock and roll, playing the blues, and everything else! It also will be an attractive decoration for your home. 【Great Beginner Guitar】Ready to use out of the box, perfect gift for Birthday,Christmas, ect. A good starter guitar kit for both kid

In [31]:
query = "I'm looking for a full-size blue electric guitar beginner kit that comes with an amplifier and accessories. Do you have something like that?"

response = qa_chain.invoke(query)

print("Question:", query)
print("\nAnswer:", response)

Question: I'm looking for a full-size blue electric guitar beginner kit that comes with an amplifier and accessories. Do you have something like that?

Answer: 

Yes, the Smartxchoices 39" Electric Guitar Beginner Kit is a full-size blue guitar with an amplifier and accessories.

**Key Features**
* All-wood construction with steel strings
* Includes a 10W amp, case and accessories
* Ultra-Thin slick design for easy control

**Specifications / Metadata**
* **Product Title:** Smartxchoices 39" Electric Guitar Beginner Kit Full Size Blue Guitar with 10W Amp, Case and Accessories Pack for Starter
* **Color:** Blue
* **Store:** Smartxchoices
* **Rating:** 4.4
* **Rating Count:** 629
* **Price:** 96.99
* **Product ID:** B082V534DD

**Summary**
This is a complete beginner guitar kit, including a 39" full-size electric guitar in blue, a 10W amplifier, a gig bag, strap, cable, strings, picks, and Allen keys. It's designed for both kids and adults.


In [37]:
query = "I'm looking for a right-handed blue electric guitar for beginners which has the lowest price."

response = qa_chain.invoke(query)

print("Question:", query)
print("\nAnswer:", response)

Question: I'm looking for a right-handed blue electric guitar for beginners which has the lowest price.

Answer: 

The lowest priced right-handed blue electric guitar for beginners is the Smartxchoices 38-inch Acoustic Guitar Package, priced at $30.99.

**Key Features**
*   Includes carrying case, guitar pick, pitch pipe, a shoulder strap, and an extra set of strings.
*   Guitar Dimensions: 38"(L) x 14"(W) x 3.25"(H)

**Specifications / Metadata**
*   **Store:** Smartxchoices
*   **Color:** Blue
*   **Rating:** 4.2
*   **Rating Count:** 2372
*   **Price:** $30.99
*   **Product ID:** B081JWTT45

**Summary**
This is a 38-inch right-handed acoustic guitar package in blue, designed for beginners, and includes essential accessories.


In [38]:
query = "I'm looking for a right-handed blue electric guitar for beginners which has the highest price."

response = qa_chain.invoke(query)

print("Question:", query)
print("\nAnswer:", response)

Question: I'm looking for a right-handed blue electric guitar for beginners which has the highest price.

Answer: 

The Smartxchoices 39’’ Electric Guitar is a right-handed blue electric guitar for beginners with a price of 96.99.

**Key Features**
* 39" Full Size Blue Electric Guitar (Right-handed)
* Ultra-Thin slick design
* Comes with a 10W amp and carry bag
* Headphone Output Connection

**Specifications / Metadata**
* **Store:** Smartxchoices
* **Color:** Blue
* **Rating:** 4.4
* **Rating Count:** 629
* **Price:** 96.99
* **Hand Orientation:** Right-handed

**Summary**
This is a functional beginner guitar made of all wood with steel strings, designed for aspiring musicians. It includes essential accessories like an amp and carry bag.


In [39]:
raw_df.iloc[0]

rating                                                         4.8
rating_count                                                100615
price                                                         6.99
store                                                   Ernie Ball
product_id                                              B0BSGM6CQ9
color                                              Mondo (10.5-52)
combined_text    Product Title: Ernie Ball Mondo Slinky Nickelw...
Name: 0, dtype: object

In [40]:
print(raw_df.iloc[0].combined_text)

Product Title: Ernie Ball Mondo Slinky Nickelwound Electric Guitar Strings 10.5-52 Gauge
Categories: Musical Instruments, Instrument Accessories, Guitar & Bass Accessories, Strings, Electric Guitar Strings
Features: Ernie Ball Slinkys are played by legends around the world including Slash, Jimmy Page, Metallica, Eric Clapton, and more Mondo Slinkys are ideal for players who prefer a heavier gauge with a little less tension on the plain strings (E, B, G) Element Shield Packaging prolongs string life and keeps strings as fresh as the day they were made Bright, balanced tone Made in California, USA with the finest and freshest materials
Description: Product Description: Ernie Ball Mondo Slinkys are ideal for players who prefer a heavier gauge with a little less tension on the plain strings (E, B, G). These electric guitar strings are precision manufactured to the highest standards and most exacting specifications to ensure consistency, optimum performance, and long life. Mondo Slinky woun

In [42]:
query = "Is there any Ernie Ball electric guitar which has 100000+ rating count?"

response = qa_chain.invoke(query)

print("Question:", query)
print("\nAnswer:", response)

Question: Is there any Ernie Ball electric guitar which has 100000+ rating count?

Answer: 

Yes, the Ernie Ball Mondo Slinky Nickelwound Electric Guitar Strings 10.5-52 Gauge has a rating count of 100615.

**Key Features**
* Ernie Ball Slinkys are played by legends.
* Ideal for players who prefer a heavier gauge with less tension.
* Element Shield Packaging prolongs string life.
* Bright, balanced tone.

**Specifications / Metadata**
* **Store:** Ernie Ball
* **Color:** Mondo (10.5-52)
* **Rating:** 4.8
* **Rating Count:** 100615
* **Price:** 6.99 
* **Product ID:** B0BSGM6CQ9

**Summary**
The Ernie Ball Mondo Slinky Nickelwound Electric Guitar Strings are a popular choice among players, boasting over 100,000 ratings and a bright, balanced tone. They are nickelwound and have a 10.5-52 gauge.


In [23]:
raw_df

,rating,rating_count,price,store,product_id,color,combined_text
0,4.8,100615,6.99,Ernie Ball,B0BSGM6CQ9,Mondo (10.5-52),Product Title: Ernie Ball Mondo Slinky Nickelw...
1,4.4,79425,29.99,BONAOK,B09W4F2X6S,Black Gold,Product Title: BONAOK Wireless Bluetooth Karao...
2,4.0,68708,14.95,BOYA,B076B8G5D8,Black,Product Title: Boya BYM1 by Shotgun Video Micr...
3,4.7,60133,10.99,D'Addario,B0BTC9YJ2W,Bronze,Product Title: D'Addario Guitar Strings - Phos...
4,4.8,55452,17.75,Amazon Basics,B018FCZKR2,Not Specified,Product Title: Amazon Basics Adjustable Guitar...
...,...,...,...,...,...,...,...
3729,4.4,548,31.99,Shamanicshop,B017NGQWA0,Brown,Product Title: Jew's Harp by P.Potkin in woode...
3730,4.3,548,13.32,Musiclily,B00WHCNE0M,Gold,Product Title: Musiclily 42mm Guitar Metal Loc...
3731,4.3,548,18.99,SumDirect,B00M35GOP0,White,Product Title: SumDirect 9 LED Music Stand Lig...
3732,4.6,547,189.99,BOSS,B086QNCLQ8,Silver,Product Title: Boss SYB-5 Bass Synthesizer Ped...


In [25]:
raw_df.store.value_counts()[:20]

store
JIM DUNLOP               97
Fender                   82
Pyle                     73
Donner                   55
D'Addario                54
YAMAHA                   52
Ernie Ball               50
Behringer                44
Musiclily                41
MusicNomad               35
OnStage                  34
Neewer                   33
Shure                    29
BOSS                     27
Korg                     26
D'Addario Accessories    26
Vandoren                 26
Gator                    26
ChromaCast               23
Hohner Accordions        22
Name: count, dtype: int64

In [43]:
query = "I want to have a YAMAHA electric guitar which is the most rated product in the site."

response = qa_chain.invoke(query)

print("Question:", query)
print("\nAnswer:", response)

Question: I want to have a YAMAHA electric guitar which is the most rated product in the site.

Answer: 
The Yamaha FG800 is the most highly rated Yamaha electric guitar in the provided information, with a rating of 4.7 based on 1151 ratings.

**Key Features**
*   Solid sitka spruce top
*   Nato back & sides
*   Rosewood fingerboard
*   Adjustable truss rod

**Specifications / Metadata**
*   **Product Title:** YAMAHA FG800 Solid Top Acoustic Guitar,Natural,Guitar Only
*   **Store:** YAMAHA
*   **Color:** Natural
*   **Rating:** 4.7
*   **Rating Count:** 1151
*   **Price:** 229.99
*   **Categories:** Musical Instruments, Guitars, Acoustic Guitars, Steel-String Acoustics

**Summary**
The Yamaha FG800 is a solid top acoustic guitar designed for quality and tone, and is recommended for beginner guitarists.
